In [20]:
import tensorflow as tf
import keras
from keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
IMAGE_SIZE=256
CHANNELS=3
BATCH_SIZE=32

In [15]:

gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip =True,
    rotation_range=10
)

train_gen = gen.flow_from_directory(
    "./data/dataset/train",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode = 'sparse',
    #save_to_dir = "./data/dataset/aug-images"
)

val_gen = gen.flow_from_directory(
    "./data/dataset/val",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode = 'sparse',
    #save_to_dir = "./data/dataset/aug-images"
)

test_gen = gen.flow_from_directory(
    "./data/dataset/test",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode = 'sparse',
    #save_to_dir = "./data/dataset/aug-images"
)


Found 1506 images belonging to 3 classes.
Found 215 images belonging to 3 classes.
Found 431 images belonging to 3 classes.


In [10]:
for image_batch, label_batch in train_gen:
    print(image_batch.shape)
    break

(32, 256, 256, 3)


In [17]:
n_classes=3
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax')
])

2024-10-25 09:17:49.772062: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-25 09:17:49.884258: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-25 09:17:49.885450: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [18]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │    15,745,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,764,675 (60.14 MB)

 Trainable params: 15,764,675 (60.14 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(
    optimizer='adam',
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [22]:
model.fit(
    train_gen,
    steps_per_epoch=47,
    batch_size=32,
    validation_data=val_gen,
    validation_steps=6,
    verbose=1,
    epochs=5
)

/home/nav/Projects/pytrain/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5


I0000 00:00:1729828350.056994 3790406 service.cc:145] XLA service 0x7c2398008da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729828350.057040 3790406 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce 940MX, Compute Capability 5.0
2024-10-25 09:22:30.125312: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-25 09:22:30.537140: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-10-25 09:22:35.474742: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[32,32,254,254]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,256,256]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation

 4/47 ━━━━━━━━━━━━━━━━━━━━ 1:01 1s/step - accuracy: 0.3978 - loss: 7.1918

I0000 00:00:1729828410.756326 3790407 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_1', 4 bytes spill stores, 12 bytes spill loads



...

47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 306ms/step - accuracy: 0.8438 - loss: 0.5056 - val_accuracy: 0.8696 - val_loss: 0.3333
Epoch 3/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.8846 - loss: 0.3038 - val_accuracy: 0.9010 - val_loss: 0.2307
Epoch 4/5


2024-10-25 09:25:57.993406: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-25 09:25:57.993471: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2024-10-25 09:25:57.993497: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8212037575635048825
2024-10-25 09:25:57.993619: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14147455388036002218


 1/47 ━━━━━━━━━━━━━━━━━━━━ 1:05 1s/step - accuracy: 0.9688 - loss: 0.1492

2024-10-25 09:25:59.394213: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-25 09:25:59.394358: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9688 - loss: 0.1492 - val_accuracy: 0.8261 - val_loss: 0.3410
Epoch 5/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.9430 - loss: 0.1469 - val_accuracy: 0.9323 - val_loss: 0.1698
